# Event finder & raw data checker

This notebook will find competitive cellular "events" in the simplest definition (i.e. a loser cell apoptosis) and return information about the spatiotemporal distrubition of counterpart competitive events (i.e. winner cell mitosis)

Contents:

- Load modules
- Load cell finding functions
- Set experiment data path
- Load image data
- Load tracking data
- Apply necessary coordinate shift for viewer
- Isolate one track of interest (target track)
- Find corresponding tracks/events of interest within a given spatiotemporal range (E.g. if target track is Scr apoptosis, then find all nearby wild-type mitosis)
- Set points and regions of interest for highlighting in Napari viewer(now set as functions so this bit is automatic
- Launch Napari image viewer if desired

To-do:

- [x] Reorganise structure so that scan over all R then filter
- [x] Create radial bins and sort tracks into them
- [ ] Create temporal bins
- [ ] Figure out how to plot said bins into heat map image
- [ ] Check redesigned script against raw data 

In [30]:
import napari
import btrack
import numpy as np
from skimage.io import imread
import os
print("Napari version no.:", napari.__version__)
print("btrack version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON, tracks_to_napari
from tqdm.notebook import tnrange, tqdm
import matplotlib.pyplot as plt

Napari version no.: 0.4.0
btrack version no.: 0.4.0


### Functions to measure local neighbourhood for cells of interest

In [31]:
def euclidean_distance(target_track, other_track, frame):
    try:
        idx0 = target_track.t.index(find_apoptosis_time(target_track, index = False)) ## could also do just ## apop_index
        idx1 = other_track.t.index(frame)
    except:
        return np.inf
    
    dx = target_track.x[idx0] - other_track.x[idx1]
    dy = target_track.y[idx0] - other_track.y[idx1]
    
    return np.sqrt(dx**2 + dy**2)

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    if index == True: 
        return apop_index
    else: 
        return apop_time

def find_nearby_wt_mitosis(target_track, delta_t, radius):
    frame = find_apoptosis_time(target_track, index = False) + delta_t 
    dividing_states = ('METAPHASE',) #('PROMETAPHASE', 'METAPHASE', 'DIVIDE')
    wt_tracks_in_radius = [wt_track for wt_track in wt_tracks if wt_track.in_frame(frame) if euclidean_distance(target_track, wt_track, frame)<radius]
    wt_mitosis_in_radius = [wt_track for wt_track in wt_tracks if wt_track.in_frame(frame) if euclidean_distance(target_track, wt_track, frame)<radius if wt_track.label[wt_track.t.index(frame)] in dividing_states if wt_track.fate.name == "DIVIDE"] ###check this
  
    return wt_tracks_in_radius, wt_mitosis_in_radius


def maximum_xyt(i):
    #### BEWARE xy coord switch!     
    if i == 'x':
        max_x = int((max([max(i.y) for i in wt_tracks])))
        return max_x
    if i == 'y':
        max_y = int((max([max(i.x) for i in wt_tracks])))
        return max_y
    if i == 't':
        max_t = int((max([max(i.t) for i in wt_tracks])))
        return max_t
    
def plot_mitoses(): ## this function plots mitosis events into the napari viewer
    apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift
    t_m, x_m, y_m = np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius))
    mito_events = np.zeros((len(wt_mitosis_in_radius), 3)) ## 3 because of the 3 cartesian coords 
    for i, wt_mitosis in enumerate(wt_mitosis_in_radius): ## this now assumes that the mitosis time point of relevance isnt the last frame of track but the time at delta_t, need to bolster definition of mitosis
        mito_index = [j for j, k in enumerate(wt_mitosis.t) if k == apop_event[0]+delta_t][0] ### [0] bc first item of list comprehension
        t_m[i], x_m[i], y_m[i] = wt_mitosis.t[mito_index], wt_mitosis.x[mito_index]+shift_y, wt_mitosis.y[mito_index]+shift_x ## plus transposed coordinate shift
        mito_events[i] = t_m[i], x_m[i], y_m[i]
    return viewer.add_points(mito_events, name = "Mitosis events", symbol = "cross", face_color = 'pink')

def plot_apoptosis(): ## this function plots apoptotic event and surrounding local environment scope (determined by radius)
    ### add_points needs to be txy
    apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift
    ## apop_circle is the bounding box of the ellipse highlighting radial extent around apoptosis, shown at the shifted time and apop time
    apop_circle = [((apop_event[0]+delta_t, apop_event[1]-radius, apop_event[2]-radius), 
                    (apop_event[0]+delta_t, apop_event[1]+radius, apop_event[2]-radius), 
                    (apop_event[0]+delta_t, apop_event[1]+radius, apop_event[2]+radius), 
                    (apop_event[0]+delta_t, apop_event[1]-radius, apop_event[2]+radius)), 
                    ((apop_event[0], apop_event[1]-radius, apop_event[2]-radius), 
                    (apop_event[0], apop_event[1]+radius, apop_event[2]-radius), 
                    (apop_event[0], apop_event[1]+radius, apop_event[2]+radius), 
                    (apop_event[0], apop_event[1]-radius, apop_event[2]+radius))]
    
    return viewer.add_points(apop_event, name = "Apoptosis event", symbol = 'x', face_color = "c"), viewer.add_shapes(apop_circle, opacity = 0.1, shape_type = 'ellipse', face_color = 'y', name = 'Radial environment')
    


### Set experiment data path 

In [32]:
# print("Input experiment number")
# experiment_no = input()
# root_path = os.path.join('/home/nathan/data/kraken/h2b/giulia/', experiment_no)
root_path = '/home/nathan/data/kraken/h2b/giulia/GV0807'  ## this overwrites input option for ease 
gfp_path = os.path.join(root_path, 'Pos3/stacks/gfp.tif')
rfp_path = os.path.join(root_path, 'Pos3/stacks/rfp.tif')
bf_path = os.path.join(root_path, 'Pos3/stacks/bf.tif')
tracks_path = os.path.join(root_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')

### Load image data

In [33]:
gfp = imread(gfp_path)

In [34]:
rfp = imread(rfp_path)

In [ ]:
#bf = imread(bf_path) #3 large 1000+ frame stacks is usually too much for memory to process on 16gb ram

### Load tracking data

In [35]:
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks
print("Tracks loaded")

### finding coord range of aligned images, coords switched already
align_x_range, align_y_range = gfp.shape[2], gfp.shape[1]
### finding maximum extent of tracking coords
tracks_x_range = round(max([max(track.x) for track in wt_tracks]))
tracks_y_range = round(max([max(track.y) for track in wt_tracks])) + 2 ## sort this lazy hack out later

### coord switch
tmp = tracks_y_range
tracks_y_range = tracks_x_range
tracks_x_range = tmp

print("tracks range:", (tracks_x_range), (tracks_y_range))
print("aligned image range:", (align_x_range), (align_y_range))

shift_x = int((align_x_range - tracks_x_range)/2)
shift_y = int((align_y_range - tracks_y_range)/2)

print("shift in x and y:", shift_x, shift_y)

wt_data, properties, graph = tracks_to_napari(wt_tracks, ndim = 2)
scr_data, properties, graph = tracks_to_napari(scr_tracks, ndim = 2)

tmp = wt_data[:,2].copy() ## copy the true_y coord
wt_data[:,2] = wt_data[:,3]  ##assign the old_y coord as the true_x
wt_data[:,3] = tmp ## assign the old_x as true_y

wt_data[:,2] += shift_y ## TRUE_Y (vertical axis)
wt_data[:,3] += shift_x ## TRUE_X (horizontal axis)

tmp = scr_data[:,2].copy()
scr_data[:,2] = scr_data[:,3]
scr_data[:,3] = tmp

scr_data[:,2] += shift_y ## TRUE_Y (vertical axis)
scr_data[:,3] += shift_x ## TRUE_X (horizontal axis)

print("coordinate shift applied")

[INFO][2020/12/11 04:47:36 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/12/11 04:47:36 PM] Loading tracks/obj_type_1
[INFO][2020/12/11 04:47:38 PM] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2020/12/11 04:47:42 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2020/12/11 04:47:42 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/12/11 04:47:42 PM] Loading tracks/obj_type_2
[INFO][2020/12/11 04:47:42 PM] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2020/12/11 04:47:42 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Tracks loaded
tracks range: 1600 1200
aligned image range: 1739 1377
shift in x and y: 69 88
coordinate shift applied


### Isolate one track of interest

In [6]:
## isolate one target scribble track of interest
print("input scribble track of interest ID")
cell_ID = int(input())
index = [i for i, scr_tracks in enumerate(scr_tracks) if scr_tracks.ID == cell_ID][0]
target_track = scr_tracks[index]##### Show the first classification of each track
apop_index, apop_time = find_apoptosis_time(target_track, index = True), find_apoptosis_time(target_track, index = False)  

input scribble track of interest ID
17


# Find cells of interest

In [ ]:
radius = 500
delta_t = -50
wt_tracks_in_radius, wt_mitosis_in_radius = find_nearby_wt_mitosis(target_track, delta_t, radius)

display ID for all tracks found in radius 

In [ ]:
print("ID for all tracks in radius:", [j.ID for i, j in enumerate(wt_tracks_in_radius)])
print("ID for all mitoses in radius:", [j.ID for i, j in enumerate(wt_mitosis_in_radius)])

to-do: 
- [x] take output of wt_tracks_in_radius and bin into discrete distance cats
- [ ] bin time periods and figure way to iterate over discrete time cats
- [ ] try new architecture based on filtering over R and T
- [ ] plot bins into heatmap

# sandbox

In [12]:
radius = 500

In [14]:
num_bins = 10
r_bin_size = radius / num_bins
i = 1

In [29]:
r_bin_size * (i)

50.0

In [22]:
cells = [wt_track for wt_track in wt_tracks if (r_bin_size * (i - 1)) < euclidean_distance(target_track, wt_track, apop_time) < (r_bin_size * i)]

In [28]:
print(cells[0])

OrderedDict([('ID', 721), ('t', [631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 82

distance binned but time not 

In [ ]:
t_range = range(-200,200)
wt_mito_ID_temporal_bins = {}

## create bins automatically
num_bins = 10
bin_size = radius / num_bins
## create dict for wt data storage
wt_track_bins = {}
wt_ID_bins = {} ## temp measure to create parallel dict just listing the IDs so that i can check algorithm works against raw data
wt_mitosis_bins = {}
wt_mito_ID_bins = {} ## temp measure to create parallel dict just listing the IDs so that i can check algorithm works against raw data

wt_tracks_in_radius = []
wt_mitosis_in_radius = []

for delta_t in t_range:
    ## creating temporal bin label
    temporal_bin_label = "frame" + str(find_apoptosis_time(target_track, index = False) + delta_t)
    wt_mito_ID_temporal_bins[temporal_bin_label] = {}
    radius = 500 ## maximum radius
    ## count all cells and mitoses within radius at delta_t
   
    wt_tracks_in_radius, wt_mitosis_in_radius = find_nearby_wt_mitosis(target_track, delta_t, radius)
    print("time", delta_t, "wt", len(wt_tracks_in_radius), "mito", len(wt_mitosis_in_radius))## currently returning zero hmmm
    ## iterate over all bins
    for i in range(num_bins):
        ## create bin label for dict entry 
        bin_label = str((bin_size * i)) + "< radial distance <" + str((bin_size * (i+1))) 
        ## create empty list to store each bins data in before storing
        wt_trackz = []
        wt_IDs = [] ## temp measure
        for wt_track in wt_tracks_in_radius:
            if (bin_size * i) <= euclidean_distance(target_track, wt_track, (find_apoptosis_time(target_track, index = False) + delta_t)) <= (bin_size*(i+1)):
                ## add track info to list for saving in dict bin entry
                wt_trackz.append(wt_track) 
                wt_IDs.append(wt_track.ID) ## temp measure
        ## add list of cells to dict entry
        wt_track_bins[bin_label] = wt_tracks
        wt_ID_bins[bin_label] = wt_IDs

        wt_mito_IDs = [] ## temp measure
        wt_mitos = []
        for wt_mito in wt_mitosis_in_radius:
            if (bin_size * i) <= euclidean_distance(target_track, wt_mito, (find_apoptosis_time(target_track, index = False) + delta_t)) <= (bin_size*(i+1)):
                ## add track info to list for saving in dict bin entry
                wt_mitos.append(wt_mito) 
                wt_mito_IDs.append(wt_mito.ID) ## temp measure
        ## add list of cells to dict entry
        wt_mitosis_bins[bin_label] = wt_mitos
        wt_mito_ID_bins[bin_label] = wt_mito_IDs
        #print(wt_mito_IDs, delta_t)
        
        #wt_mito_ID_temporal_bins[temporal_bin_label][bin_label] = wt_mito_IDs

In [ ]:
wt_mito_ID_temporal_bins

plotting wt_mito_ID_temporal_bins

In [ ]:
wt_mito_ID_temporal_bins['frame685']

In [ ]:
heatmap = np.zeros((10,400))

for i, key1 in enumerate(wt_mito_ID_temporal_bins.keys()):
    #print(i, key1)
    for j, key2 in enumerate(wt_mito_ID_temporal_bins['frame685'].keys()):
        #print(j,key2)
        if len(wt_mito_ID_temporal_bins[key1][key2]) > 4:
            print(key1, key2)
            print(len(wt_mito_ID_temporal_bins[key1][key2]))
        heatmap[j,i] = len(wt_mito_ID_temporal_bins[key1][key2])
#heatmap[:,0] = wt_mito_ID_temporal_bins[i][j]

In [ ]:
plt.imshow(heatmap, cmap= 'viridis', aspect = 'auto')#, #interpolation = 'nearest')
plt.colorbar()
plt.ylim(0,9)
#plt.xlim(0,10)
#plt.clim(0,3)
plt.title("Number of wild-type mitosis")
plt.text(195,-4.5, "Scr apoptosis -", rotation=90)
plt.xlabel("frame (unaligned)")
plt.ylabel("distance (50pixel bins)")
plt.savefig("/home/nathan/data/number_heatmap_Scr17.pdf", transparent=True, bbox_inches="tight", pad_inches=0.25, dpi = 300)
plt.show()

### Launch napari to check against raw data 

In [ ]:
with napari.gui_qt():
    viewer = napari.Viewer()
    
    #viewer.add_image(bf)
    viewer.add_image(gfp, name="gfp", blending = "additive", colormap = "green")
    viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "red")
    
    viewer.add_tracks(wt_data)
    viewer.add_tracks(scr_data)
    
    plot_mitoses() ## utilises two functions above to plot apop and mito 
    plot_apoptosis()
    viewer.dims.set_point(0, (apop_time + delta_t)) ## open viewer at the timepoint of interest